### Import Dependencies

In [1]:
import gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
import os

### Test Enviroment

In [2]:
##!pip install -U atari-py

In [3]:
##!python -m atari_py.import_roms ./ROMS

copying adventure.bin from ./ROMS/Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin to /home/joezik/.local/lib/python3.10/site-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from ./ROMS/Air Raid (Men-A-Vision) (PAL) ~.bin to /home/joezik/.local/lib/python3.10/site-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from ./ROMS/Alien (1982) (20th Century Fox Video Games, Douglas 'Dallas North' Neubauer) (11006) ~.bin to /home/joezik/.local/lib/python3.10/site-packages/atari_py/atari_roms/alien.bin
copying amidar.bin from ./ROMS/Amidar (1982) (Parker Brothers, Ed Temple) (PB5310) ~.bin to /home/joezik/.local/lib/python3.10/site-packages/atari_py/atari_roms/amidar.bin
copying assault.bin from ./ROMS/Assault (AKA Sky Alien) (1983) (Bomb - Onbase) (CA281).bin to /home/joezik/.local/lib/python3.10/site-packages/atari_py/atari_roms/assault.bin
copying asterix.bin from ./ROMS/Asterix (AKA Taz) (1984) (Atari, Jerome Domurat, Steve Woita) (CX2696).bin

In [4]:
enviroment_rame =  'Breakout-v0'
env = gym.make(enviroment_rame)

A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]


In [5]:
##env.reset()

In [6]:
##env.action_space

In [7]:
##env.observation_space

In [8]:
episodes =  5
for episode in range(1, episodes+1):
    obs =  env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

/home/joezik/.local/lib/python3.10/site-packages/gym/envs/atari/environment.py:267: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(


Episode:1 Score:1.0
Episode:2 Score:0.0
Episode:3 Score:3.0
Episode:4 Score:0.0
Episode:5 Score:4.0


### Vectorise Enviroment and Train Model

In [21]:
env = make_atari_env('Breakout-v0', n_envs=4, seed=0)
env = VecFrameStack(env, n_stack=4)

In [11]:
env.render()

True

In [12]:
env.close()

In [22]:
log_path =  os.path.join('Training', 'Logs')
model = A2C('CnnPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [33]:
model.learn(total_timesteps=300000)

Logging to Training/Logs/A2C_3
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 427      |
|    ep_rew_mean        | 4.83     |
| time/                 |          |
|    fps                | 411      |
|    iterations         | 100      |
|    time_elapsed       | 4        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -0.415   |
|    explained_variance | 0.948    |
|    learning_rate      | 0.0007   |
|    n_updates          | 5099     |
|    policy_loss        | 0.225    |
|    value_loss         | 0.128    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 450      |
|    ep_rew_mean        | 5.2      |
| time/                 |          |
|    fps                | 417      |
|    iterations         | 200      |
|    time_elapsed       | 9        |
|    total_timesteps    | 4000     |
| train

### Save and Reload Model

In [34]:
a2c_path = os.path.join('Training','Saved Models','A2C_Breakout_Model')
model.save(a2c_path)

In [35]:
del model

In [36]:
model =  A2C.load(a2c_path, env)

Wrapping the env in a VecTransposeImage.


### Evaluate and Test

##### Earlier we created 4 enviroments, but evaluate_policy only accepts an enviroment with a single instance to test the model

In [37]:
env =  make_atari_env('Breakout-v0', n_envs=1, seed=0)
env =  VecFrameStack(env, n_stack=4)

In [41]:
evaluate_policy(model, env, n_eval_episodes=100, render=True)

(12.51, 4.99298507908846)

In [42]:
env.close()